In [ ]:
from benchmarks import power_2samp_dimension
from hyppo.independence import CCA, Dcorr, HHG, Hsic, RV, MGC
from hyppo.sims import *

#sys.path.append(os.path.realpath('..'))

In [ ]:
import seaborn as sns
sns.set(color_codes=True, style='white', context='talk', font_scale=2)
PALETTE = sns.color_palette("Set1")
sns.set_palette(PALETTE[3:])

In [ ]:
import rpy2
from rpy2.robjects import Formula, numpy2ri
from rpy2.robjects.packages import importr

In [ ]:
MAX_DIMENSION = 2
STEP_SIZE = 1
DIMENSIONS = range(1, MAX_DIMENSION + STEP_SIZE, STEP_SIZE)
POWER_REPS = 1
simulations = [
    linear,
    exponential,
    cubic,
    joint_normal,
    step,
    quadratic,
    w_shaped,
    spiral,
    uncorrelated_bernoulli,
    logarithmic,
    fourth_root,
    sin_four_pi,
    sin_sixteen_pi,
    square,
    two_parabolas,
    circle,
    ellipse,
    diamond,
    multiplicative_noise,
    multimodal_independence
]

#test = perm_test(X,y)

In [ ]:
#def estimate_power(sim, test):
def estimate_power(sim):
    est_power = np.array([np.mean([power_2samp_dimension(UF, trans_2samp, sim, p=i) for _ in range(POWER_REPS)])
                          for i in DIMENSIONS])
    np.savetxt('../benchmarks/2samp_vs_dimension/{}_{}.csv'.format(sim.__name__, "UF"),
               est_power, delimiter=',')
    
    return est_power

In [ ]:
outputs = Parallel(n_jobs=-1, verbose=100)(
    #[delayed(estimate_power)(sim, perm_test(X, y)) for sim in simulations]
    [delayed(estimate_power)(sim) for sim in simulations]
)

In [ ]:
def plot_power():
    fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(28,24))
    
    sim_title = [
        "Linear",
        "Exponential",
        "Cubic",
        "Joint Normal",
        "Step",
        "Quadratic",
        "W-Shaped",
        "Spiral",
        "Bernoulli",
        "Logarithmic",
        "Fourth Root",
        "Sine 4\u03C0",
        "Sine 16\u03C0",
        "Square",
        "Two Parabolas",
        "Circle",
        "Ellipse",
        "Diamond",
        "Multiplicative",
        "Independence"
    ]

    for i, row in enumerate(ax):
        for j, col in enumerate(row): 
            count = 5*i + j
            sim = simulations[count]
            
            power = np.genfromtxt('../benchmarks/2samp_vs_dimensions/{}_{}.csv',format(sim._name_, "UF"), delimiter= ",")
            
            col.plot(DIMENSIONS, power, label=test._name_, lw=1)
            col.set_xticks([])
            if i == 3:
                    col.set_xticks([DIMENSIONS[0], DIMENSIONS[-1]])
            col.set_ylim(-1.05, 1.05)
            if j==0: 
                col.set_yticks([-1, 0, 1])
            col.set_title(sim_title[count])
            
    fig.text(0.5, 0.08, 'Dimension', ha='center')
    fig.text(0.08, 0.5, 'Absolute Statistical Power', va='center', rotation='vertical')
    leg = plt.legend(bbox_to_anchor=(0.5, 0.08), bbox_transform=plt.gcf().transFigure,
                     ncol=5, loc='upper center')
    leg.get_frame().set_linewidth(0.0)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(5.0)
    plt.subplots_adjust(hspace=.50)
    plt.savefig('../benchmarks/figs/2samp_power_dimension.pdf', transparent=True, bbox_inches='tight')

In [ ]:
plot_power()